In [104]:
# готовим датасет

import random
import csv


def f(_x1, _x2):
    return _x1 * _x1 + _x2 * _x2


x1_min = -10.0
x2_min = -10.0
x1_max = 10.0
x2_max = 10.0

with open('data/regression_2/regression_2.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["x1", "x2", "y"])
    for i in range(0, 1000):
        x1 = random.uniform(x1_min, x1_max)
        x2 = random.uniform(x2_min, x2_max)
        # y = f(x1, x2) + random.uniform(-0.5, 0.5)
        y = f(x1, x2)
        writer.writerow([x1, x2, y])


In [105]:
# выделяем тренировочный и проверочный датасеты

import csv
import numpy as np

data = []
with open('data/regression_2/regression_2.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for row in reader:
        float_row = list(map(float, row))
        data.append(float_row)

npdata = np.array(data)
total = len(npdata)

train_size = int(0.7 * total)
test_size = int(0.3 * total)

index_1 = 0
index_2 = train_size
index_3 = train_size + test_size

train_x = npdata[index_1:index_2, :2]
train_y = npdata[index_1:index_2, 2:]

test_x = npdata[index_2:index_3, :2]
test_y = npdata[index_2:index_3, 2:]

print("data", len(data))
print("train", len(train_x), len(train_y))
print("test", len(test_x), len(test_y))

data 1000
train 700 700
test 300 300


In [106]:
import tensorflow as tf
from matplotlib import pyplot as plt

tf.reset_default_graph()

with tf.name_scope("input"):
    x = tf.placeholder(dtype=tf.float32, shape=(None, 2), name="x")
    y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="y")

features_count = 7

with tf.name_scope("features"):
    feature_1 = tf.slice(x, [0, 0], [-1, 1], name="feature_x1")
    feature_2 = tf.slice(x, [0, 1], [-1, 1], name="feature_x2")
    feature_3 = tf.multiply(feature_1, feature_2, name="feature_x1_x2")
    feature_4 = tf.multiply(feature_1, feature_1, name="feature_x1_x1")
    feature_5 = tf.multiply(feature_2, feature_2, name="feature_x2_x2")
    # feature_6 = tf.sin(feature_1, name="feature_sin_x1")
    # feature_7 = tf.sin(feature_2, name="feature_sin_x2")

with tf.name_scope("layer_1"):
    w1 = tf.Variable(0, dtype=tf.float32)
    m1 = tf.multiply(feature_1, w1)
    w2 = tf.Variable(0, dtype=tf.float32)
    m2 = tf.multiply(feature_2, w2)
    w3 = tf.Variable(0, dtype=tf.float32)
    m3 = tf.multiply(feature_3, w3)
    w4 = tf.Variable(1, dtype=tf.float32)
    m4 = tf.multiply(feature_4, w4)
    w5 = tf.Variable(1, dtype=tf.float32)
    m5 = tf.multiply(feature_5, w5)
    # w6 = tf.Variable(0, dtype=tf.float32)
    # m6 = tf.multiply(feature_6, w6)
    # w7 = tf.Variable(0, dtype=tf.float32)
    # m7 = tf.multiply(feature_7, w7)
    b = tf.Variable(0, dtype=tf.float32)

# prediction = tf.add(m1 + m2 + m3 + m4 + m5 + m6 + m7, b, "output/prediction")
prediction = tf.add(m1 + m2, b, "output/prediction")
error = tf.reduce_mean(tf.square(prediction - y), name="output/error")

points = [[], []]
learning_rate = tf.Variable(0.0005, dtype=tf.float32, name="learning_rate")
epochs = 20000
optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate=learning_rate).minimize(
    error,
    # var_list=[w1, w2, w3, w4, w5, w6, w7, b]
    var_list=[w1, w2, w3, w4, b]
)

with tf.Session() as sess:
    learning_rate.initializer.run()
    w1.initializer.run()
    w2.initializer.run()
    w3.initializer.run()
    w4.initializer.run()
    w5.initializer.run()
    # w6.initializer.run()
    # w7.initializer.run()
    b.initializer.run()

    train_writer = tf.summary.FileWriter('logs/dnn/train', sess.graph)

    feed_dict = {
        x: train_x,
        y: train_y
    }
    last_error = None
    for i in list(range(epochs)):
        result = sess.run(fetches=[optimizer, prediction, error], feed_dict=feed_dict)
        current_error = result[2]
        if i % 10 == 0.:
            points[0].append(i + 1)
            points[1].append(current_error)
        if i % 100 == 0:
            print(i, current_error)
        if last_error is not None and abs(last_error - current_error) < 1:
            learning_rate.assign_sub(0.00001)
        last_error = current_error
    plt.plot(points[0], points[1], 'r--')
    plt.show()

# print("output", result[1])


0 5581.6157
100 4813.5156
200 4216.5186
300 3727.274
400 3326.2996
500 2997.6704


600 2728.3347
700 2507.5918
800 2326.677
900 2178.4023
1000 2056.8801


1100 1957.2831


1200 1875.6555


1300 1808.7555


KeyboardInterrupt: 